In [1]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf
from tqdm import tqdm

In [2]:
import torch
import keras.backend as K

In [3]:
from transformers import AutoTokenizer, AutoModel, BertConfig

In [4]:
# bert_model = "vinai/bertweet-base"
bert_model = 'bert-base-uncased'
# bert_model = 'google/electra-small-discriminator'

tokenizer = AutoTokenizer.from_pretrained(bert_model)
bert = AutoModel.from_pretrained(bert_model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
train_data = []
train_label = []
train = pd.read_csv('data/train/train_enrich.csv', delimiter = '\t')
train

,text,label
0,5. Can regularly rinsing your nose with saline...,0
1,5. Can regularly rinsing your nose with saline...,0
2,5. Can regularly rinsing your nose with saline...,0
3,5. Can regularly rinsing your nose with saline...,0
4,5. Can regularly rinsing your nose with saline...,0
...,...,...
30414,@rosierawle Tasha and I just little giggle at...,0
30415,@Kirstenjoyweiss @prestone85 @ldsknack I beli...,0
30416,@rosierawle All good. Just been working out t...,0
30417,@Kirstenjoyweiss @prestone85 @ldsknack Even t...,0


In [5]:
dev_data = []
dev_label = []
dev = pd.read_csv('data/dev/dev_enrich.csv', delimiter = '\t')
dev

,text,label
0,COVID-19 Fact:\nAre hand dryers effective in k...,0
1,COVID-19 Fact:\nAre hand dryers effective in k...,0
2,@atruchecks when can we expect the result of m...,0
3,@atruchecks when can we expect the result of m...,0
4,@atruchecks when can we expect the result of m...,0
...,...,...
10549,"19 (including drugs, vaccines, therapies)?\n\...",0
10550,"19 (including drugs, vaccines, therapies)?\n\...",0
10551,"19 (including drugs, vaccines, therapies)?\n\...",0
10552,"19 (including drugs, vaccines, therapies)?\n\...",0


In [89]:
for i in tqdm(range(len(dev))):
    text = dev.iloc[i]['text']
    label = dev.iloc[i]['label']
    dev_label.append(label)
    s = tokenizer(text, padding='max_length', truncation=True)
    dev_data.append(bert(torch.tensor((s['input_ids'])).unsqueeze(0), attention_mask = torch.tensor((s['attention_mask'])).unsqueeze(0)).last_hidden_state.tolist()[0][0])

100%|██████████| 10554/10554 [34:40<00:00,  5.07it/s] 


In [194]:
for i in tqdm(range(len(train))):
    text = train.iloc[i]['text']
    label = train.iloc[i]['label']
    train_label.append(label)
    s = tokenizer(text, padding='max_length', truncation=True)
    train_data.append(bert(torch.tensor((s['input_ids'])).unsqueeze(0), attention_mask = torch.tensor((s['attention_mask'])).unsqueeze(0)).last_hidden_state.tolist()[0][0])

100%|██████████| 30419/30419 [11:34:55<00:00,  1.37s/it]  


In [54]:
for i in tqdm(range(len(train))):
    label = train.iloc[i]['label']
    train_label.append(label)

100%|██████████| 30419/30419 [00:02<00:00, 13155.91it/s]


In [27]:
import csv
train_file = open('data/train/train_basebert.csv', 'w', newline = '')
train_dataset = csv.writer(train_file)
train_dataset.writerows(train_data)
train_file.close()

In [90]:
# import csv
# dev_file = open('data/dev/dev_matrix.csv', 'w', newline = '')
# dev_dataset = csv.writer(dev_file)
# dev_dataset.writerows(dev_data)
# dev_file.close()

In [6]:
import csv
train_data = []
train_file = open('data/train/train_matrix.csv', 'r', newline = '')
train_dataset = csv.reader(train_file)
for row in train_dataset:
    train_data.append(row)
train_file.close()

In [7]:
import csv
dev_data = []
dev_file = open('data/dev/dev_matrix.csv', 'r', newline = '')
dev_dataset = csv.reader(dev_file)
for row in dev_dataset:
    dev_data.append(row)
dev_file.close()

In [8]:
x_dev = np.array(dev_data).astype(float)
# y_dev = np.array(dev_label).astype(int)

In [9]:
x = np.array(train_data)
x = x.astype(float)
x.dtype

dtype('float64')

In [10]:
index = 0
y_train = []
# f = open('data/dev/dev_pred.txt', 'w')
dict = train['source_id'].value_counts()
x_train = []
for i in range(1895):
    x_train.append(x[index:index+dict[i]])
    y_train.append(train['label'][index])
    index += dict[i]
# f.close()

KeyError: 'source_id'

In [35]:
x_train_64 = []
for row in x_train:
    if len(row) >= 64:
        x_train_64.append(row[:64])
    else:
        temp = np.concatenate((row,[[0] * 768] * (64-len(row))))
        x_train_64.append(temp)

In [42]:
x = np.array(x_train_64)
x = x.astype(float)
x.shape

(1895, 64, 768)

In [43]:
y_train = np.array(y_train).astype(int)
y_train.shape

(1895,)

In [52]:
index = 0
y_dev = []
# f = open('data/dev/dev_pred.txt', 'w')
dict = dev['source_id'].value_counts()
x_devl = []
for i in range(632):
    x_devl.append(x_dev[index:index+dict[i]])
    y_dev.append(dev['label'][index])
    index += dict[i]
# f.close()

In [53]:
x_dev_64 = []
for row in x_devl:
    if len(row) >= 64:
        x_dev_64.append(row[:64])
    else:
        temp = np.concatenate((row,[[0] * 768] * (64-len(row))))
        x_dev_64.append(temp)
x_dev = np.array(x_dev_64)
x_dev = x_dev.astype(float)
x_dev.shape
y_dev = np.array(y_dev).astype(int)
y_dev.shape

(632,)

In [22]:
y = np.array(train['label']).astype(int)
y_dev = np.array(dev['label']).astype(int)

In [56]:
y = np.array(train_label)
y.shape

(30419,)

In [57]:
y = y.astype(int)

In [27]:
x.shape

(30419, 768)

In [188]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_dim = 768),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [69]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
from keras.layers import *
from keras.models import *

In [28]:
model = Sequential()
# model.add(Embedding(vocab_size, 300, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [29]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [30]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy', get_f1])

In [31]:
history = model.fit(x, y, epochs=10, batch_size = 128, validation_data=(x_dev,y_dev))

Epoch 1/10


ValueError: in user code:

    File "C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Miniconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\input_spec.py", line 228, in assert_input_compatibility
        raise ValueError(f'Input {input_index}

In [72]:
y_pred = history.model.predict(x_dev)

In [76]:
y_pred[:10]

array([[0.12654859],
       [0.02063759],
       [0.02118009],
       [0.09812991],
       [0.05099511],
       [0.0104217 ],
       [0.01235239],
       [0.5913856 ],
       [0.03979163],
       [0.6963214 ]], dtype=float32)

In [73]:
true_label = []
true_label_f = open('data/dev/dev.label.txt', 'r')
for line in true_label_f.readlines():
    true_label.append(line.strip('\n'))
true_label_f.close()
true_label[:10]

['nonrumour',
 'nonrumour',
 'nonrumour',
 'nonrumour',
 'nonrumour',
 'nonrumour',
 'nonrumour',
 'nonrumour',
 'nonrumour',
 'nonrumour']

In [74]:
index = 0
p_label = []
# f = open('data/dev/dev_pred.txt', 'w')
dict = dev['source_id'].value_counts()
for i in range(632):
    if sum(y_pred[index:index+dict[i], 0])/dict[i] > 0.5:
    # for p in y_pred[index:index+dict[i], 0]:
    #     p_vote = [0, 0]
    #     if p > 0.5:
    #         p_vote[0] += 1
    #     else:
    #         p_vote[1] += 1
    # if p_vote[0] > p_vote[1]:    
        p_label.append('rumour')
        # f.write('rumour\n')
    else:
        p_label.append('nonrumour')
        # f.write('nonrumour\n')
    index += dict[i]
# f.close()

In [75]:
from sklearn.metrics import f1_score

f1_score(true_label, p_label, pos_label='rumour', average='binary')

0.0